<h2>Importando as bibliotecas necessárias

In [13]:
import requests as rq
import pandas as pd

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

from io import BytesIO
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import numpy as np

<ipython-input-13-f8cda1096f69>:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)  # or 199


<h2>Criando o dataframe das planilhas hospedadas no github

In [14]:
url = "https://github.com/kairobeduti/DadosShift/blob/master/Mensagens/Async_Api_Notes.xlsx?raw=true"
data = rq.get(url).content
df = pd.read_excel(BytesIO(data), engine='openpyxl')

In [15]:
url = "https://github.com/kairobeduti/DadosShift/blob/master/An%C3%A1lise/Principal.xlsx?raw=true"
data = rq.get(url).content
df_principal = pd.read_excel(BytesIO(data), engine='openpyxl')

In [16]:
df.set_axis(['seq','tdformat','tdline','oportunidade'],axis=1,inplace=True)
df.head()

,seq,tdformat,tdline,oportunidade
0,0,NaN,<b>Note 1 : </b>Oportunidadecriadaemduplicidade.,7001980909
1,0,NaN,<b>Note 1 : </b>CriadaemDuplicidade.,7001980938
2,0,X,"<b>Note 1 : </b>PAGAMENTO PARCELADO\n\n1. MEDIÇÃO DOS SERVIÇOS\n\n1.1 - A PETROBRAS procederá à medição dos serviços executados, reuni",7001981014
3,1,X,"encontrados do Relatório de Pequenos Serviços (RPS),assinado por ambas as partes e entregue à CONTRATADA no 1º (primeiro)dia útil",7001981014
4,2,X,"do mês seguinte ao da medição, para fins de apresentação dosdocumentos de cobrança.\n\n1.2 - O período de execução dos serviços a ser",7001981014


In [17]:
oportunidades = df.oportunidade.unique().tolist()

In [18]:
lista_concats = []
for n in oportunidades:
    df_slice = df[df['oportunidade']==n]
    str_final = ''
    tp = ()
    for x,t in df_slice.iterrows():
        str_final += ' '+str(t['tdline'])
    lista_concats.append((n,str_final))
df_textos = pd.DataFrame(lista_concats,columns=['oportunidade','texto'])

In [19]:
import texthero as hero
import nltk

# Clean text
df_textos["texto"] = df_textos["texto"].pipe(hero.clean)
df_textos["tfidf"] = (hero.tfidf(df_textos["texto"], max_features=100))
df_textos["tfidf"].head(2)

text = " ".join(df_textos["texto"].values)

In [28]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords.append('br')
stopwords.append('note')
stopwords.append('de')
stopwords.append('b')
stopwords.append('e')

In [29]:
custom_stopwords = set(stopwords).union(set(stopwords))

In [31]:
a = hero.remove_stopwords(df_textos["texto"],custom_stopwords)
hero.top_words(a)

licitacao                                                                                                                               5808
nao                                                                                                                                     5071
item                                                                                                                                    4662
conforme                                                                                                                                4046
propostas                                                                                                                               3960
oportunidade                                                                                                                            3424
comissao                                                                                                                                3254
servicos     

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

%matplotlib inline

def make_wordcloud(new_text):
    """'function to make wordcloud"""

    wordcloud = WordCloud(
        stopwords=stopwords,
        width=1500,
        height=1500,
        min_font_size=1,
        background_color="black",
        colormap="Set2",
        collocation_threshold=90,
    ).generate(new_text)

    fig = plt.figure(figsize=(12, 12), facecolor=None)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.tight_layout(pad=0)

    plt.show()

    return fig

text_cloud = make_wordcloud(text)